In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, avg, count, sum, max

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("COVID19_Metadata_SQL_Analytics") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .getOrCreate()

optimized_parquet_path = "hdfs:///covid_dataset/metadata_optimized/"

# Читаем данные напрямую из папки Parquet в HDFS
df_sql = spark.read.parquet(optimized_parquet_path)

# Создаем SQL-таблицу из DataFrame
df_sql.createOrReplaceTempView("covid_metadata")
print("Выполнение аналитических SQL-запросов")

# Запрос 1: Распределение диагнозов и средний возраст по полу
query1_result = spark.sql("""
    SELECT
        finding_unified,
        sex,
        COUNT(*) AS total_cases,
        ROUND(AVG(age_numeric), 2) AS avg_age
    FROM
        covid_metadata
    GROUP BY
        finding_unified, sex
    ORDER BY
        finding_unified, total_cases DESC
""")

print("Запрос 1: Распределение диагнозов и средний возраст по полу")
query1_result.show(truncate=False)

# Запрос 2: ТОП-5 локаций по количеству случаев COVID-19 и средняя температура
query2_result = spark.sql("""
    SELECT
        location,
        COUNT(*) AS covid_cases_count,
        ROUND(AVG(temperature), 2) AS avg_temperature
    FROM
        covid_metadata
    WHERE
        is_covid = 1 AND location IS NOT NULL AND TRIM(location) != '' AND LOWER(location) != 'null'
    GROUP BY
        location
    ORDER BY
        covid_cases_count DESC
    LIMIT 5
""")

print("Запрос 2: ТОП-5 локаций по количеству случаев COVID-19 и средняя температура")
query2_result.show(truncate=False)

# Запрос 3: Динамика случаев COVID-19 по годам и месяцам с использованием оконных функций
query3_result = spark.sql("""
    SELECT
        year,
        month,
        COUNT(*) AS monthly_covid_cases,
        SUM(COUNT(*)) OVER (ORDER BY year, month ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_covid_cases
    FROM
        covid_metadata
    WHERE
        is_covid = 1 AND year IS NOT NULL AND month IS NOT NULL
    GROUP BY
        year, month
    ORDER BY
        year, month
""")

print("Запрос 3: Динамика случаев COVID-19 по годам и месяцам (оконная функция)")
query3_result.show(truncate=False)


# Запрос 4: Сложный запрос с самосоединением (Self-Join). Уникальные пары пациентов с общим диагнозом/полом, но разным возрастом
query4_result = spark.sql("""
    WITH PairedPatients AS (
        SELECT DISTINCT
            a.patientid AS patient1_id,
            a.age_numeric AS patient1_age,
            b.patientid AS patient2_id,
            b.age_numeric AS patient2_age,
            a.finding_unified,
            a.sex,
            ABS(a.age_numeric - b.age_numeric) AS age_difference -- Вычисляем разницу здесь
        FROM
            covid_metadata a
        JOIN
            covid_metadata b ON a.finding_unified = b.finding_unified
                             AND a.sex = b.sex
                             AND a.patientid < b.patientid
        WHERE
            ABS(a.age_numeric - b.age_numeric) > 10
    )
    SELECT
        patient1_id,
        patient1_age,
        patient2_id,
        patient2_age,
        finding_unified,
        sex
    FROM
        PairedPatients
    ORDER BY
        finding_unified, age_difference DESC -- Теперь сортируем по новой колонке
    LIMIT 10
""")

print("Запрос 4: Сложный запрос с самосоединением (Self-Join). Уникальные пары пациентов с общим диагнозом/полом, но разным возрастом.")
query4_result.show(truncate=False)


# Запрос 5: Аналитический подзапрос для нахождения местоположения с наибольшей средней температурой
query5_result = spark.sql("""
    SELECT
        location,
        covid_cases_count,
        avg_temperature
    FROM (
        SELECT
            location,
            COUNT(*) AS covid_cases_count,
            ROUND(AVG(temperature), 2) AS avg_temperature,
            ROW_NUMBER() OVER (ORDER BY AVG(temperature) DESC) AS rn
        FROM
            covid_metadata
        WHERE
            is_covid = 1 AND location IS NOT NULL AND TRIM(location) != '' AND LOWER(location) != 'null' AND temperature IS NOT NULL
        GROUP BY
            location
        HAVING
            COUNT(*) > 10
    ) subquery
    WHERE
        rn = 1
""")

print("Запрос 5: Аналитический подзапрос - локация с самой высокой средней температурой среди активных")
query5_result.show(truncate=False)

# Запрос 6: Агрегация с условием и оконная функция (процент случаев COVID-19 от общего числа в каждом месяце)
query6_result = spark.sql("""
    SELECT
        year,
        month,
        COUNT(patientid) AS total_monthly_cases,
        COUNT(CASE WHEN is_covid = 1 THEN patientid ELSE NULL END) AS monthly_covid_cases,
        ROUND(
            (COUNT(CASE WHEN is_covid = 1 THEN patientid ELSE NULL END) * 100.0) / COUNT(patientid), 2
        ) AS percentage_covid_cases_of_month
    FROM
        covid_metadata
    WHERE
        year IS NOT NULL AND month IS NOT NULL
    GROUP BY
        year, month
    ORDER BY
        year, month
""")

print("Запрос 6: Агрегация и процент COVID-случаев от общего числа в каждом месяце")
query6_result.show(truncate=False)


# Запрос 7: Оконная функция для расчета скользящего среднего возраста по месяцам
query7_result = spark.sql("""
    SELECT
        year,
        month,
        ROUND(AVG(age_numeric), 2) AS monthly_avg_age,
        ROUND(AVG(AVG(age_numeric)) OVER (ORDER BY year, month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS moving_avg_age_3_months
    FROM
        covid_metadata
    WHERE
        is_covid = 1 AND age_numeric IS NOT NULL AND year IS NOT NULL AND month IS NOT NULL
    GROUP BY
        year, month
    ORDER BY
        year, month
""")

print("Запрос 7: Оконная функция - скользящее среднее возраста пациентов с COVID-19")
query7_result.show(truncate=False)


# Запрос 8: Сложная агрегация: средняя сатурация кислорода и минимальная температура по группе возраста и виду
query8_result = spark.sql("""
    SELECT
        age_group,
        view,
        COUNT(patientid) AS total_patients,
        ROUND(AVG(pO2_saturation), 2) AS avg_pO2_saturation,
        ROUND(MIN(temperature), 2) AS min_temperature
    FROM
        covid_metadata
    WHERE
        age_group != 'unknown' AND pO2_saturation IS NOT NULL AND temperature IS NOT NULL
    GROUP BY
        age_group, view
    ORDER BY
        age_group, total_patients DESC
""")

print("Запрос 8: Сложная агрегация - средняя сатурация и мин. температура по возрастной группе и типу снимка")
query8_result.show(truncate=False)


spark.stop()
print("SparkSession остановлена. Аналитика завершена.")

Выполнение аналитических SQL-запросов
Запрос 1: Распределение диагнозов и средний возраст по полу
+---------------+---+-----------+-------+
|finding_unified|sex|total_cases|avg_age|
+---------------+---+-----------+-------+
|covid-19       |M  |409        |51.29  |
|covid-19       |F  |175        |55.54  |
|no finding     |M  |12         |45.5   |
|no finding     |F  |10         |58.0   |
|other finding  |M  |61         |40.0   |
|other finding  |F  |23         |40.0   |
|other pneumonia|M  |144        |47.68  |
|other pneumonia|F  |98         |49.39  |
|tuberculosis   |M  |13         |46.92  |
|tuberculosis   |F  |5          |30.2   |
+---------------+---+-----------+-------+

Запрос 2: ТОП-5 локаций по количеству случаев COVID-19 и средняя температура
+---------------------------------------------------------------+-----------------+---------------+
|location                                                       |covid_cases_count|avg_temperature|
+---------------------------------------------------------------+-----------------+---------------+
|Italy                                                          |79               |38.18          |
|Hannover Medical School, Hannover, Germany                     |79               |38.17          |
|Spain                                                          |18               |38.16          |
|United Kingdom                                                 |18               |38.17          |
|Humanitas Clinical and Research Hospital, Rozzano, Milan, Italy|17               |38.14          |
+---------------------------------------------------------------+-----------------+---------------+

Запрос 3: Динамика случаев COVID-19 по годам и месяцам (оконная функция)
+----+-----+-------------------+----------------------+
|year|month|monthly_covid_cases|cumulative_covid_cases|
+----+-----+-------------------+----------------------+
|20  |1    |1                  |1                     |
|20  |3    |3                  |4                     |
|2019|12   |4                  |8                     |
|2020|1    |393                |401                   |
|2020|2    |20                 |421                   |
|2020|3    |30                 |451                   |
|2020|4    |1                  |452                   |
+----+-----+-------------------+----------------------+

Запрос 4: Сложный запрос с самосоединением (Self-Join). Уникальные пары пациентов с общим диагнозом/полом, но разным возрастом.
                                                                                
+-----------+------------+-----------+------------+---------------+---+
|patient1_id|patient1_age|patient2_id|patient2_age|finding_unified|sex|
+-----------+------------+-----------+------------+---------------+---+
|326b       |94          |471        |20          |covid-19       |M  |
|314        |21          |326b       |94          |covid-19       |M  |
|217        |22          |326b       |94          |covid-19       |M  |
|326b       |94          |354        |25          |covid-19       |M  |
|133        |25          |326b       |94          |covid-19       |M  |
|326b       |94          |476        |25          |covid-19       |M  |
|324b       |93          |358        |25          |covid-19       |F  |
|324b       |93          |69         |25          |covid-19       |F  |
|200        |88          |471        |20          |covid-19       |M  |
|210        |27          |326b       |94          |covid-19       |M  |
+-----------+------------+-----------+------------+---------------+---+

Запрос 5: Аналитический подзапрос - локация с самой высокой средней температурой среди активных
                                                                                
+---------+-----------------+---------------+
|location |covid_cases_count|avg_temperature|
+---------+-----------------+---------------+
|Hong Kong|14               |38.27          |
+---------+-----------------+---------------+

Запрос 6: Агрегация и процент COVID-случаев от общего числа в каждом месяце
                                                                                
+----+-----+-------------------+-------------------+-------------------------------+
|year|month|total_monthly_cases|monthly_covid_cases|percentage_covid_cases_of_month|
+----+-----+-------------------+-------------------+-------------------------------+
|12  |1    |1                  |0                  |0.00                           |
|19  |11   |2                  |0                  |0.00                           |
|20  |1    |1                  |1                  |100.00                         |
|20  |3    |5                  |3                  |60.00                          |
|2004|1    |11                 |0                  |0.00                           |
|2007|1    |1                  |0                  |0.00                           |
|2009|9    |3                  |0                  |0.00                           |
|2010|1    |3                  |0                  |0.00                           |
|2010|5    |2                  |0                  |0.00                           |
|2010|10   |1                  |0                  |0.00                           |
|2011|1    |5                  |0                  |0.00                           |
|2013|1    |7                  |0                  |0.00                           |
|2014|1    |11                 |0                  |0.00                           |
|2015|1    |24                 |0                  |0.00                           |
|2015|5    |1                  |0                  |0.00                           |
|2016|1    |20                 |0                  |0.00                           |
|2017|1    |3                  |0                  |0.00                           |
|2017|6    |1                  |0                  |0.00                           |
|2018|1    |6                  |0                  |0.00                           |
|2019|1    |2                  |0                  |0.00                           |
+----+-----+-------------------+-------------------+-------------------------------+
only showing top 20 rows

Запрос 7: Оконная функция - скользящее среднее возраста пациентов с COVID-19
[Stage 23:================================================>       (12 + 2) / 14]
+----+-----+---------------+-----------------------+
|year|month|monthly_avg_age|moving_avg_age_3_months|
+----+-----+---------------+-----------------------+
|20  |1    |31.0           |31.0                   |
|20  |3    |50.0           |40.5                   |
|2019|12   |52.5           |44.5                   |
|2020|1    |51.2           |51.23                  |
|2020|2    |55.8           |53.17                  |
|2020|3    |56.77          |54.59                  |
|2020|4    |71.0           |61.19                  |
+----+-----+---------------+-----------------------+

Запрос 8: Сложная агрегация - средняя сатурация и мин. температура по возрастной группе и типу снимка
                                                                                
+---------+---------+--------------+------------------+---------------+
|age_group|view     |total_patients|avg_pO2_saturation|min_temperature|
+---------+---------+--------------+------------------+---------------+
|adult    |PA       |268           |89.14             |36.0           |
|adult    |AP Supine|192           |88.61             |36.8           |
|adult    |AP       |129           |88.94             |36.4           |
|adult    |L        |66            |89.34             |38.0           |
|adult    |Axial    |61            |88.89             |37.8           |
|adult    |Coronal  |14            |88.99             |38.17          |
|adult    |AP Erect |1             |88.99             |38.17          |
|senior   |PA       |76            |89.2              |36.5           |
|senior   |AP       |74            |88.77             |36.1           |
|senior   |AP Supine|42            |89.61             |37.1           |
|senior   |L        |18            |88.71             |38.17          |
|senior   |Axial    |7             |88.99             |38.17          |
|senior   |Coronal  |2             |88.99             |38.17          |
+---------+---------+--------------+------------------+---------------+

SparkSession остановлена. Аналитика завершена.